# Price Prediction of used cars - Pakwheels

-----------


## Import Libraries

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')



## Load Dataset

In [39]:
df = pd.read_csv('D:/DS Bootcamp/Machine Learning/Used cars Price prediction/Used-Cars-Price-Prediction-Pakwheels/Pakwheels_used_cars.csv')
df.head()

,title,location,model_year,fuel_type,transmission,km_driven,registered_in,color,assembly,engine_capacity,body_type,last_updated,price,url
0,Suzuki Mehran 2003,Multan Punjab,2003.0,Petrol,Manual,"170,000 km",Lahore,White,Local,800 cc,NaN,"May 07, 2025",PKR 5.8 Lakh,https://www.pakwheels.com/used-cars/suzuki-meh...
1,Mitsubishi Lancer GLX Automatic 1.6 2006,Lahore Punjab,2006.0,Petrol,Automatic,"119,080 km",Islamabad,Labrador Black Pearl,Imported,1600 cc,Sedan,"Apr 29, 2025",PKR 25 Lakh,https://www.pakwheels.com/used-cars/mitsubishi...
2,Suzuki Wagon R Stingray X 2013,"Abbot Road, Lahore Punjab",2013.0,Petrol,Automatic,"87,000 km",Punjab,Black,Imported,660 cc,Hatchback,"Apr 29, 2025",PKR 26.5 Lakh,https://www.pakwheels.com/used-cars/suzuki-wag...
3,Toyota Land Cruiser ZX 2019,Lahore Punjab,2019.0,Petrol,Automatic,"30,000 km",Un-Registered,White Pearl Crystal Shine,Imported,4600 cc,SUV,"Apr 28, 2025",PKR 4.63 crore,https://www.pakwheels.com/used-cars/toyota-lan...
4,Suzuki Alto VXL AGS 2021,"Rehmania Town, Faisalabad Punjab",2021.0,Petrol,Automatic,"15,800 km",Punjab,Silver,Local,660 cc,Hatchback,"Apr 30, 2025",PKR 27.7 Lakh,https://www.pakwheels.com/used-cars/suzuki-alt...


------

## Data Preprocessing

### Handle Data inconsisties

In [40]:
pd.set_option('display.max_rows', False)

From title column extract the brand name and store it to a new column called brand

In [41]:
df.insert(0, 'brand', df['title'].str.split().str[0])

From title remove the model yr bcz we already have year of model in dataset


In [43]:
df['title'] = df['title'].str.replace(r'\b\d{4}$', '', regex=True).str.strip()

Rename the title to model_name

In [44]:
df.rename(columns={'title':'model_name'},inplace=True)

Extract from location all the provinces and stored in new column province

In [49]:
df.insert(2, 'province', df['location'].str.split().str[-1])

Correct the dtype of model_year into int.

In [58]:
df['model_year'] = pd.to_numeric(df['model_year'], errors='coerce')

Replace comma and km from the km_driven column

In [68]:
df['km_driven'] = df['km_driven'].replace({',': '', ' km': ''}, regex=True).str.strip()

Correct the dtype of km_driven into int

In [70]:
df['km_driven'] = pd.to_numeric(df['km_driven'], errors='coerce')

In [69]:
df['km_driven'].value_counts()

km_driven
100000    407
150000    225
200000    198
80000     158
90000     133
50000     125
120000    113
70000     108
85000     103
1         103
60000      99
10000      96
         ... 
25467       1
103333      1
13900       1
119886      1
79          1
140725      1
110846      1
6708        1
108592      1
1122        1
145352      1
93994       1
Name: count, Length: 2182, dtype: int64

In [71]:
df

,brand,model_name,province,location,model_year,fuel_type,transmission,km_driven,registered_in,color,assembly,engine_capacity,body_type,last_updated,price,url
0,Suzuki,Suzuki Mehran,Punjab,Multan Punjab,2003,Petrol,Manual,170000.0,Lahore,White,Local,800 cc,NaN,"May 07, 2025",PKR 5.8 Lakh,https://www.pakwheels.com/used-cars/suzuki-meh...
1,Mitsubishi,Mitsubishi Lancer GLX Automatic 1.6,Punjab,Lahore Punjab,2006,Petrol,Automatic,119080.0,Islamabad,Labrador Black Pearl,Imported,1600 cc,Sedan,"Apr 29, 2025",PKR 25 Lakh,https://www.pakwheels.com/used-cars/mitsubishi...
2,Suzuki,Suzuki Wagon R Stingray X,Punjab,"Abbot Road, Lahore Punjab",2013,Petrol,Automatic,87000.0,Punjab,Black,Imported,660 cc,Hatchback,"Apr 29, 2025",PKR 26.5 Lakh,https://www.pakwheels.com/used-cars/suzuki-wag...
3,Toyota,Toyota Land Cruiser ZX,Punjab,Lahore Punjab,2019,Petrol,Automatic,30000.0,Un-Registered,White Pearl Crystal Shine,Imported,4600 cc,SUV,"Apr 28, 2025",PKR 4.63 crore,https://www.pakwheels.com/used-cars/toyota-lan...
4,Suzuki,Suzuki Alto VXL AGS,Punjab,"Rehmania Town, Faisalabad Punjab",2021,Petrol,Automatic,15800.0,Punjab,Silver,Local,660 cc,Hatchback,"Apr 30, 2025",PKR 27.7 Lakh,https://www.pakwheels.com/used-cars/suzuki-alt...
5,MG,MG ZS 1.5L,Punjab,"Johar Town, Lahore Punjab",2021,Petrol,Automatic,85556.0,Punjab,White,Imported,1490 cc,Compact SUV,"May 07, 2025",PKR 47 Lakh,https://www.pakwheels.com/used-cars/mg-zs-2021...
6,Suzuki,Suzuki Wagon R VXL,Islamabad,"G- 9, Islamabad Islamabad",2018,Petrol,Manual,78000.0,Islamabad,Solid White,Local,1000 cc,Hatchback,"May 04, 2025",PKR 23.7 Lakh,https://www.pakwheels.com/used-cars/suzuki-wag...
7,Toyota,Toyota Fortuner GR-S,Punjab,"Sharqi Colony, Vehari Punjab",<NA>,Diesel,Automatic,30000.0,Punjab,Super White,Local,2800 cc,SUV,"May 07, 2025",PKR 2.08 crore,https://www.pakwheels.com/used-cars/toyota-for...
8,Suzuki,Suzuki Wagon R VXL,Punjab,"G.T Road, Gujrat Punjab",2018,Petrol,Manual,114000.0,Lahore,White,Local,1000 cc,Hatchback,"Apr 24, 2025",PKR 21.5 Lakh,https://www.pakwheels.com/used-cars/suzuki-wag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [60]:
for i in df:
    print(df[i].nunique())
    print(df[i].value_counts())
    print('==============================================')

56
brand
Toyota        2995
Suzuki        2942
Honda         1753
Daihatsu       556
Nissan         284
Hyundai        226
KIA            226
Mitsubishi     143
Mercedes       127
Haval           95
Changan         83
Audi            69
              ... 
GUGO             1
Volvo            1
Golden           1
JW               1
Hummer           1
Fiat             1
SsangYong        1
Power            1
Deepal           1
Daehan           1
Datsun           1
Hino             1
Name: count, Length: 56, dtype: int64
1267
model_name
Suzuki Alto VXR                               336
Toyota Corolla GLi 1.3 VVTi                   221
Suzuki Mehran VXR Euro II                     209
Honda Civic Oriel 1.8 i-VTEC CVT              206
Honda City 1.3 i-VTEC                         165
Suzuki Cultus VXR                             158
Suzuki Mehran VX                              142
Suzuki Alto VXL AGS                           142
Suzuki Wagon R VXL                            138
Toyota Corol

In [72]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   brand            9985 non-null   object 
 1   model_name       9985 non-null   object 
 2   province         9985 non-null   object 
 3   location         9985 non-null   object 
 4   model_year       8938 non-null   Int64  
 5   fuel_type        9985 non-null   object 
 6   transmission     9985 non-null   object 
 7   km_driven        9985 non-null   float64
 8   registered_in    9985 non-null   object 
 9   color            9985 non-null   object 
 10  assembly         9985 non-null   object 
 11  engine_capacity  9985 non-null   object 
 12  body_type        8932 non-null   object 
 13  last_updated     9985 non-null   object 
 14  price            9985 non-null   object 
 15  url              10000 non-null  object 
dtypes: Int64(1), float64(1), object(14)
memory usage: 1.2+ MB


,model_year,km_driven
count,8938.0,9985.000000
mean,2013.425375,92642.012919
std,7.319762,88467.153004
min,1990.0,1.000000
25%,2008.0,37380.000000
50%,2015.0,80000.000000
75%,2020.0,123568.000000
max,2022.0,1000000.000000
